# YOLOv8 Training and Evaluation for Fish Detection


## 1. Setup Environment & Data Preparation
- Đường dẫn ảnh và label
- Chia train/val
- Tạo file data.yaml
- Kiểm tra số lượng ảnh và lớp


In [1]:
!pip install ultralytics
from ultralytics import YOLO
import os
import cv2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/AI_Fish_Classification/data_fish_in_real_time/filtered_images.zip \
      -d /content/fish_data

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: /content/fish_data/filtered_images/Syngnathidae/59eafd18-3e6a-4569-bdfa-5bcdac4497a1.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5a55b468-8cb1-4e19-ab6f-9cda555aec8c.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5a71de03-b8bb-4a54-b089-1762755965ea.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5acd3221-9dd5-4f64-a266-3d1cf85a3da5.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5ace4cc3-2992-4a03-b039-a6d39ccc1868.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5ada0612-4e42-4dde-8c89-5d254c30d5a1.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5b19edf1-6ef1-479b-bbf1-e93f6bae5dd7.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5b2ca458-85bd-41cf-bc5a-4bad300fd824.jpg  
  inflating: /content/fish_data/filtered_images/Syngnathidae/5b37c780-de91-488e-a47a-d56c02893af7.jpg  
  infla

In [4]:
!unzip /content/drive/MyDrive/AI_Fish_Classification/code/YOLO/bbox.zip \
      -d /content/data_labels

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
 extracting: /content/data_labels/all_family/Odontaspididae/0c41fc1a-2fbe-4fef-b448-13e0a7e29798.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/8e185faa-f9fc-4b6f-ab04-70aaa021f4b9.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/c2fb4d2b-dee5-45c1-b215-1f1533bb94e4.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/86f3181f-ce32-4af1-a368-97e56493618c.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/833888bf-9000-4108-b922-21f9a305c259.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/7a2a0016-d215-4f39-87a6-e7f0d6b3fa24.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/0e23e7b7-a2df-4672-aa65-cc58875d6ae2.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/0f4cce5d-9314-4eaa-96d1-a870f7faadd1.txt  
 extracting: /content/data_labels/all_family/Odontaspididae/7474f7b4-70d7-42c4-ae7a-7d2e6453488b.txt  
 extracting: /co

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split
import cv2
import yaml
from concurrent.futures import ThreadPoolExecutor

# ---- PATH ----
base_path = "/content/YOLO"  # RAM Colab
image_root = "/content/fish_data/filtered_images"  # folder ảnh đã unzip
label_root = "/content/data_labels/all_family"     # folder label bbox
yaml_path = os.path.join(base_path, "data.yaml")

# ---- 1. Lấy danh sách tất cả ảnh ----
image_files = []
for family in os.listdir(image_root):
    family_path = os.path.join(image_root, family)
    if os.path.isdir(family_path):
        for f in os.listdir(family_path):
            if f.lower().endswith((".jpg", ".png", ".jpeg")):
                image_files.append((family, os.path.join(family_path, f)))

# ---- 2. Lấy danh sách class ----
families = sorted(list(set(f for f, _ in image_files)))
class_map = {name: idx for idx, name in enumerate(families)}
print("Class map:", class_map)

# ---- 3. Chia train/val/test (70/15/15) ----
image_names = [path for _, path in image_files]
train_imgs, temp_imgs = train_test_split(image_names, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)
splits = {"train": train_imgs, "val": val_imgs, "test": test_imgs}

# ---- 4. Tạo folder YOLO chuẩn trong RAM ----
for split in ["train", "val", "test"]:
    img_dir = os.path.join(base_path, "images", split)
    lbl_dir = os.path.join(base_path, "labels", split)
    if os.path.exists(img_dir):
        shutil.rmtree(img_dir)
    if os.path.exists(lbl_dir):
        shutil.rmtree(lbl_dir)
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(base_path, "images", split), exist_ok=True)
    os.makedirs(os.path.join(base_path, "labels", split), exist_ok=True)

# ---- 5. Hàm copy ảnh + tạo nhãn YOLO ----
def process_image(img_path, split):
    family_name = os.path.basename(os.path.dirname(img_path))
    img_name = os.path.splitext(os.path.basename(img_path))[0]

    # Copy ảnh
    dst_img = os.path.join(base_path, "images", split, os.path.basename(img_path))
    shutil.copy(img_path, dst_img)

    # Nhãn gốc
    label_src = os.path.join(label_root, family_name, img_name + ".txt")
    if not os.path.exists(label_src):
        print("Không tìm thấy file nhãn:", label_src)
        return

    # Lấy kích thước ảnh
    img_cv = cv2.imread(img_path)
    h, w = img_cv.shape[:2]

    # Tạo nhãn YOLO
    label_dst = os.path.join(base_path, "labels", split, img_name + ".txt")
    with open(label_src, "r") as f_in, open(label_dst, "w") as f_out:
        for line in f_in:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            fam, x_min, y_min, width, height = parts
            x_min, y_min, width, height = map(float, [x_min, y_min, width, height])
            class_id = class_map[fam]
            x_center = (x_min + width / 2) / w
            y_center = (y_min + height / 2) / h
            w_norm = width / w
            h_norm = height / h
            f_out.write(f"{class_id} {x_center} {y_center} {w_norm} {h_norm}\n")

# ---- 6. Chạy song song với ThreadPool ----
for split, imgs in splits.items():
    with ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(lambda p: process_image(p, split), imgs)

# ---- 7. Tạo data.yaml ----
data_yaml = {
    "train": os.path.join(base_path, "images", "train"),
    "val": os.path.join(base_path, "images", "val"),
    "test": os.path.join(base_path, "images", "test"),
    "nc": len(families),
    "names": families
}
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("Dataset YOLO train/val/test đã sẵn sàng trong RAM")


Class map: {'Acanthuridae': 0, 'Acestrorhynchidae': 1, 'Acheilognathidae': 2, 'Achiridae': 3, 'Acipenseridae': 4, 'Acropomatidae': 5, 'Adrianichthyidae': 6, 'Aetobatidae': 7, 'Agonidae': 8, 'Ailiidae': 9, 'Akysidae': 10, 'Albulidae': 11, 'Alepisauridae': 12, 'Alepocephalidae': 13, 'Alestidae': 14, 'Alopiidae': 15, 'Ambassidae': 16, 'Amblycipitidae': 17, 'Amblyopsidae': 18, 'Amiidae': 19, 'Ammodytidae': 20, 'Amphiliidae': 21, 'Anabantidae': 22, 'Anablepidae': 23, 'Anacanthobatidae': 24, 'Anarhichadidae': 25, 'Anguillidae': 26, 'Anomalopidae': 27, 'Anoplogastridae': 28, 'Anoplopomatidae': 29, 'Anostomidae': 30, 'Antennariidae': 31, 'Antigoniidae': 32, 'Aphaniidae': 33, 'Apistidae': 34, 'Aploactinidae': 35, 'Aplocheilidae': 36, 'Aplodactylidae': 37, 'Apogonidae': 38, 'Apteronotidae': 39, 'Aracanidae': 40, 'Argentinidae': 41, 'Arhynchobatidae': 42, 'Ariidae': 43, 'Ariommatidae': 44, 'Arripidae': 45, 'Artedidraconidae': 46, 'Aspredinidae': 47, 'Astroblepidae': 48, 'Ateleopodidae': 49, 'Athe

In [7]:
# in số lượng ảnh có trong mỗi tập
base_path = "/content/YOLO/images"

for split in ["train", "val", "test"]:
    folder = os.path.join(base_path, split)
    count = len([f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".png", ".jpeg"))])
    print(split, ":", count)


train : 59276
val : 12702
test : 12702


## 2. Loading & Training the Model
- Thiết lập epochs, batch size, image size
- Đường dẫn lưu kết quả
- Theo dõi training loss


In [ ]:
from ultralytics import YOLO
import os

# ---- PATH ----
base_path = "/content/drive/MyDrive/AI_Fish_Classification/code/YOLO"
yaml_path = os.path.join(base_path, "data.yaml")  # data.yaml vẫn lưu trên Drive
save_dir = os.path.join(base_path, "runs/train/fish_yolo")  # nơi lưu checkpoint

# ---- TẠO FOLDER SAVE NẾU CHƯA CÓ ----
os.makedirs(save_dir, exist_ok=True)

# ---- KIỂM TRA CHECKPOINT ----
# Nếu có best.pt hoặc last.pt, sẽ dùng để tiếp tục train
checkpoint_path = None
best_path = os.path.join(save_dir, "weights/best.pt")
last_path = os.path.join(save_dir, "weights/last.pt")

if os.path.exists(best_path):
    checkpoint_path = best_path
elif os.path.exists(last_path):
    checkpoint_path = last_path

# ---- LOAD MODEL ----
if checkpoint_path:
    print(f"Tiếp tục huấn luyện từ checkpoint: {checkpoint_path}")
    model = YOLO(checkpoint_path)
else:
    print("Huấn luyện mới từ yolov8n.pt")
    model = YOLO("yolov8n.pt")

# ---- TRAIN ----
# Chú ý: ảnh và nhãn đã được copy vào RAM trong code tạo dataset trước đó
model.train(
    data=yaml_path,       # vẫn dùng data.yaml trên Drive
    epochs=20,            # số epoch sẽ cộng tiếp với checkpoint
    batch=32,
    imgsz=512,
    device=0,             # GPU 0
    project=os.path.join(base_path, "runs/train"),
    name="fish_yolo_new",
    exist_ok=True
)


In [ ]:
# Tiếp tục train từ checkpoint cũ
!yolo train model="/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/runs/train/fish_yolo/weights/last.pt" data="/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/data.yaml" project="/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/runs/train" name="fish_yolo" resume=True


## 3. Evaluation
- Đánh giá trên validation set
- Precision, Recall, mAP
- Confusion matrix
- Đánh giá trên các nhóm dữ liệu

### Đánh giá trên tập val

In [8]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/runs/train/fish_yolo_new/weights/best_new.pt"
model = YOLO(model_path)

metrics = model.val(data="/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/data.yaml", device=0, verbose=True)

# Lấy dict kết quả trung bình
res = metrics.results_dict

print("===== VALIDATION METRICS (ALL CLASSES) =====")
print(f"Precision : {res['metrics/precision(B)']:.3f}")
print(f"Recall    : {res['metrics/recall(B)']:.3f}")
print(f"mAP50     : {res['metrics/mAP50(B)']:.3f}")
print(f"mAP50-95  : {res['metrics/mAP50-95(B)']:.3f}")

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,450,713 parameters, 0 gradients, 10.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 689.8±509.8 MB/s, size: 49.7 KB)
val: Scanning /content/YOLO/labels/val... 12702 images, 0 backgrounds, 424 corrupt: 100% ━━━━━━━━━━━━ 12702/12702 950.9it/s 13.4s
val: /content/YOLO/images/val/000cbf4c-d890-4821-9bcb-0ee654fbe5e8.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.054]
val: /content/YOLO/images/val/00ede875-946d-4e34-b361-cf8d72d71270.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.129       1.164      1.3423       1.186      1.3468]
val: /content/YOLO/images/val/01112881-a070-46ec-ae1c-323e2b51783a.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.111]
val: /content/YOLO/images/val/02711e07-86d2-40e7-9f4d-7556d23966f6.jpg: ignoring corrupt image/

### Đánh giá trên tập test

In [ ]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/runs/train/fish_yolo_new/weights/best_new.pt"
model = YOLO(model_path)

# Đánh giá trên tập test
metrics = model.val(data="/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/data.yaml", device=0, split='test', verbose=True)

# Lấy dict kết quả trung bình
res = metrics.results_dict

print("===== TEST METRICS (ALL CLASSES) =====")
print(f"Precision : {res['metrics/precision(B)']:.3f}")
print(f"Recall    : {res['metrics/recall(B)']:.3f}")
print(f"mAP50     : {res['metrics/mAP50(B)']:.3f}")
print(f"mAP50-95  : {res['metrics/mAP50-95(B)']:.3f}")

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,450,713 parameters, 0 gradients, 10.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 139.1±253.3 MB/s, size: 66.0 KB)
val: Scanning /content/YOLO/labels/test.cache... 12702 images, 0 backgrounds, 423 corrupt: 100% ━━━━━━━━━━━━ 12702/12702 10.5Mit/s 0.0s
val: /content/YOLO/images/test/00150f4a-4557-4d33-a1f5-8cd058a7e8a3.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.335       1.177       1.188]
val: /content/YOLO/images/test/002c6b2b-a3ac-4fe1-b21d-3787fed5be8d.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.154      1.2107]
val: /content/YOLO/images/test/007e2e51-e376-4c67-81f8-8b792d5b27b0.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.181]
val: /content/YOLO/images/test/03160bee-51a4-4f31-89b7-23ab066e599d.jpg: ignoring corrupt image/la

### Chia bộ dữ liệu test thành các nhóm theo độ phổ biến

In [24]:
import os
import shutil
import os
from ultralytics import YOLO
import yaml

# --- Tạo folder test theo nhóm ---
group_base = "/content/YOLO/test_groups"
for group in ["common", "medium", "rare"]:
    img_dir = os.path.join(group_base, "images", group)
    lbl_dir = os.path.join(group_base, "labels", group)
    if os.path.exists(img_dir):
        shutil.rmtree(img_dir)
    if os.path.exists(lbl_dir):
        shutil.rmtree(lbl_dir)
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(lbl_dir, exist_ok=True)

# --- Copy ảnh + nhãn vào folder nhóm ---
def copy_group_images(imgs, families_group, group_name):
    for img_path in imgs:
        family_name = os.path.basename(os.path.dirname(img_path))
        if family_name in families_group:
            img_name = os.path.basename(img_path)
            lbl_name = os.path.splitext(img_name)[0] + ".txt"

            # Copy ảnh
            dst_img = os.path.join(group_base, "images", group_name, img_name)
            shutil.copy(img_path, dst_img)

            # Copy nhãn
            src_lbl = os.path.join(label_root, family_name, lbl_name)
            if os.path.exists(src_lbl):
                dst_lbl = os.path.join(group_base, "labels", group_name, lbl_name)
                shutil.copy(src_lbl, dst_lbl)

copy_group_images(test_imgs, common_families, "common")
copy_group_images(test_imgs, medium_families, "medium")
copy_group_images(test_imgs, rare_families, "rare")
# ---- PATHS ----
group_base = "/content/YOLO/test_groups"
yaml_path = "/content/YOLO/data.yaml"  # file data.yaml gốc

# ---- Load danh sách class từ YAML ----
import yaml
with open(yaml_path, "r") as f:
    data_cfg = yaml.safe_load(f)

names = data_cfg["names"]  # danh sách tên loài
name_to_id = {name: idx for idx, name in enumerate(names)}

# ---- Hàm chuyển đổi nhãn ----
def convert_labels(group_name):
    lbl_dir = os.path.join(group_base, group_name, "labels")
    img_dir = os.path.join(group_base, group_name, "images")

    for lbl_file in os.listdir(lbl_dir):
        lbl_path = os.path.join(lbl_dir, lbl_file)
        img_name = os.path.splitext(lbl_file)[0] + ".jpg"
        img_path = os.path.join(img_dir, img_name)

        # Lấy kích thước ảnh
        import cv2
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Không đọc được ảnh {img_path}")
            continue
        h, w = img.shape[:2]

        new_lines = []
        with open(lbl_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"⚠️ Sai định dạng trong {lbl_file}: {line}")
                    continue

                cls_name, xmin, ymin, xmax, ymax = parts
                if cls_name not in name_to_id:
                    print(f"⚠️ Không tìm thấy class {cls_name} trong YAML")
                    continue

                cls_id = name_to_id[cls_name]
                xmin, ymin, xmax, ymax = map(float, [xmin, ymin, xmax, ymax])

                # Chuyển sang YOLO format
                x_center = ((xmin + xmax) / 2) / w
                y_center = ((ymin + ymax) / 2) / h
                width = (xmax - xmin) / w
                height = (ymax - ymin) / h

                new_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # Ghi đè file nhãn
        with open(lbl_path, "w") as f:
            f.write("\n".join(new_lines))

    print(f"✅ Đã chuyển đổi nhãn cho nhóm {group_name}")

# ---- Chạy cho cả 3 nhóm ----
for group in ["common", "medium", "rare"]:
    convert_labels(group)


✅ Đã chuyển đổi nhãn cho nhóm common
✅ Đã chuyển đổi nhãn cho nhóm medium
✅ Đã chuyển đổi nhãn cho nhóm rare


### Đánh giá trên data common

In [25]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/runs/train/fish_yolo_new/weights/best_new.pt"
model = YOLO(model_path)

# Đánh giá trên tập test
metrics = model.val(data="/content/YOLO/data_common.yaml", device=0, split='test', verbose=True)

# Lấy dict kết quả trung bình
res = metrics.results_dict

print("===== METRICS (COMMON) =====")
print(f"Precision : {res['metrics/precision(B)']:.3f}")
print(f"Recall    : {res['metrics/recall(B)']:.3f}")
print(f"mAP50     : {res['metrics/mAP50(B)']:.3f}")
print(f"mAP50-95  : {res['metrics/mAP50-95(B)']:.3f}")

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,450,713 parameters, 0 gradients, 10.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1487.8±565.0 MB/s, size: 45.8 KB)
val: Scanning /content/YOLO/test_groups/common/labels... 2808 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2808/2808 2.5Kit/s 1.1s
val: New cache created: /content/YOLO/test_groups/common/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 176/176 9.1it/s 19.4s
                   all       2808       3086      0.721      0.518       0.58      0.269
         Centrarchidae        450        455      0.645      0.563      0.561      0.196
            Cyprinidae        539        628      0.684      0.369      0.446      0.174
              Gobiidae        445        458      0.905       0.74      0.831      0.384
              Labridae        812        887      0.705   

### Đánh giá trên data medium

In [26]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/runs/train/fish_yolo_new/weights/best_new.pt"
model = YOLO(model_path)

# Đánh giá trên tập test
metrics = model.val(data="/content/YOLO/data_medium.yaml", device=0, split='test', verbose=True)

# Lấy dict kết quả trung bình
res = metrics.results_dict

print("===== TEST METRICS (ALL CLASSES) =====")
print(f"Precision : {res['metrics/precision(B)']:.3f}")
print(f"Recall    : {res['metrics/recall(B)']:.3f}")
print(f"mAP50     : {res['metrics/mAP50(B)']:.3f}")
print(f"mAP50-95  : {res['metrics/mAP50-95(B)']:.3f}")

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,450,713 parameters, 0 gradients, 10.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1260.3±281.8 MB/s, size: 52.8 KB)
val: Scanning /content/YOLO/test_groups/medium/labels... 6711 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 6711/6711 2.5Kit/s 2.7s
val: New cache created: /content/YOLO/test_groups/medium/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 420/420 9.6it/s 44.0s
                   all       6711       7518      0.581      0.461       0.48      0.219
          Acanthuridae        201        233      0.423      0.438      0.423      0.209
            Apogonidae        160        177      0.869      0.808      0.843      0.448
               Ariidae         47         51      0.733      0.255      0.389      0.185
          Aulostomidae         78         84      0.535   

### Đánh giá trên data rare

In [27]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/AI_Fish_Classification/code/YOLO/runs/train/fish_yolo_new/weights/best_new.pt"
model = YOLO(model_path)

# Đánh giá trên tập test
metrics = model.val(data="/content/YOLO/data_rare.yaml", device=0, split='test', verbose=True)

# Lấy dict kết quả trung bình
res = metrics.results_dict

print("===== TEST METRICS (ALL CLASSES) =====")
print(f"Precision : {res['metrics/precision(B)']:.3f}")
print(f"Recall    : {res['metrics/recall(B)']:.3f}")
print(f"mAP50     : {res['metrics/mAP50(B)']:.3f}")
print(f"mAP50-95  : {res['metrics/mAP50-95(B)']:.3f}")

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,450,713 parameters, 0 gradients, 10.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1090.1±287.2 MB/s, size: 32.3 KB)
val: Scanning /content/YOLO/test_groups/rare/labels... 3183 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3183/3183 2.3Kit/s 1.4s
val: New cache created: /content/YOLO/test_groups/rare/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 199/199 9.6it/s 20.8s
                   all       3183       3433      0.558      0.369      0.399      0.224
     Acestrorhynchidae          5          5          1          0      0.272     0.0863
      Acheilognathidae          9          9      0.859      0.889      0.889      0.515
             Achiridae          4          4      0.461        0.5      0.524      0.368
         Acipenseridae         10         14      0.583      0